## Part 1: Data Cleanup
To begin with, we will first look at the data to see what features / columns we should keep. 

Here in our case, we will use the most recent 1.5 million record from the Chicago crime dataset instead of using the whole dataset (which contains 6 million records back to year 2001. This will covers the crime data back from year 2011. We choose not to use earlier data because the distribution of crimes can change over time due to various reasons, and the most recent 4 to 5 years will be more relevant to the current situation (if we view the problem as a classification problem instead of a time-series problem).

We will clean up the data as below:
1. Throw away extremely rare crime types and locations. We understand that this step may introduce some bias, and having this step may not significantly improve our classification accuracy. However, this step reduces the number of features we input for the classification algorithms, and also reduces the types of crime we need to classify our data into (when doing multiclassfication).
2. Throw away rows with vague information (for example "others") or typo or NA.
3. Convert categorical features such as crime type and police district to dummy (indicator) variables.
4. Extract the time information (and convert categorical features such as weekdays to dummy variables).
5. Merge the data with weather data.
6. Add a feature "distance to closest police station".
7. Include several features describing the income, housing condition, education level and population structure of the area where the crime takes place.

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
data_small=pd.read_csv("small_chicago_crime_data.csv")

In [3]:
data_small.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10313623,HY503033,11/15/2015 11:51:00 PM,069XX S DAMEN AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,True,False,735,7,17,67,16,1164219,1858818,2015,11/22/2015 03:55:53 PM,41.768214,-87.673617,"(41.768213543, -87.673617146)"
1,10313594,HY503010,11/15/2015 11:46:00 PM,041XX W 47TH ST,0560,ASSAULT,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),True,False,815,8,14,57,08A,1149632,1873148,2015,11/22/2015 03:55:53 PM,41.807832,-87.726715,"(41.807831884, -87.726715402)"
2,10313635,HY503005,11/15/2015 11:40:00 PM,012XX N HONORE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,True,True,1424,14,1,24,08B,1163925,1908232,2015,11/22/2015 03:55:53 PM,41.903817,-87.673303,"(41.903816925, -87.673303026)"
3,10313604,HY503001,11/15/2015 11:40:00 PM,059XX S CALUMET AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,ALLEY,False,True,232,2,20,40,14,1179499,1865673,2015,11/22/2015 03:55:53 PM,41.786689,-87.617400,"(41.786689088, -87.617399787)"
4,10314701,HY503763,11/15/2015 11:30:00 PM,026XX N BOSWORTH AVE,0810,THEFT,OVER $500,STREET,False,False,1932,19,32,7,06,1165540,1917725,2015,11/22/2015 03:55:53 PM,41.929832,-87.667100,"(41.929832059, -87.667099876)"


We can see there are a lot of information, but some columns are not necessary at least for now. For example, we can delete "Case Number", "Domestic", "Beat", "Updated on", "Arrest". Some columns will be directly relevant to our classification, for example the "Date", "Primary Type", "Location Description", "Community Area", "Latitude", "Longitude". Some other columns may not be directly relevant, but we will keep them as well (and delete them later if we need to do classification based on specific columns).

In [4]:
data_small=data_small.drop(["Case Number", "Domestic", "Beat", "Updated On", "Arrest"], 1)

Though we hope to classify every single type of crime, it is not really practical since some crimes are happening at extremely low frequency (for example less than 10 records in all 6 million records). Therefore, we will first find out the most frequent types of crime.

In [5]:
datagb_crime=data_small.groupby("Primary Type")["Primary Type"].count()
datagb_crime.order(ascending=False, inplace=True)
datagb_crime.head(50)

Primary Type
THEFT                                334255
BATTERY                              267471
CRIMINAL DAMAGE                      157383
NARCOTICS                            157237
BURGLARY                              93524
ASSAULT                               90603
OTHER OFFENSE                         88338
MOTOR VEHICLE THEFT                   67535
DECEPTIVE PRACTICE                    66123
ROBBERY                               57565
CRIMINAL TRESPASS                     38378
WEAPONS VIOLATION                     17237
PUBLIC PEACE VIOLATION                14362
OFFENSE INVOLVING CHILDREN            10922
PROSTITUTION                           9170
CRIM SEXUAL ASSAULT                    6492
INTERFERENCE WITH PUBLIC OFFICER       6152
SEX OFFENSE                            4783
GAMBLING                               2758
LIQUOR LAW VIOLATION                   2335
HOMICIDE                               2211
ARSON                                  2139
KIDNAPPING         

Based on the sorted list of crimes, we can see that some crimes are really rare, for example human trafficking. Though these rare records won't greatly affect our model (they might slightly reduce the accuracy of our prediction), we will only pick top 25 types of crime for simplicity.

In [6]:
crime_list=datagb_crime.index.values[0:25].tolist()
data_small=data_small[data_small["Primary Type"].isin(crime_list)]
#data_small.groupby("Primary Type")["Primary Type"].count()

Looking at the top 20 types of crimes, we actually need to do more clean up. The definition of crimes are listed here:

http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html

We can see "other offense", which accounts for around 6% of the total crime records, involve lots of different categories of crime that cannot be categorized easily. Therefore we drop the rows with other offense.

In [7]:
data_small=data_small[data_small["Primary Type"]!="OTHER OFFENSE"]

In our situation, the final response variable will be "Primary Type", which corresponds to 19 types of crimes. However, we can also manually categorize these 19 types of crime into "severe" and "not-severe" to make a new response variable, so that we can start our classification from a "yes-or-no" model to determine whether a crime is going to be severe (with regards to violence and aftermath) or not. We will classify the following types of crimes as severe (and give them an indicator variable 1): "ARSON", "ASSAULT", "BATTERY", "CRIM SEXUAL ASSAULT", "CRIMINAL DAMAGE", "CRIMINAL TRESPASS", "HOMICIDE", "ROBBERY". 

We want to emphasize that "severe" variable in our case only means that the crime involves direct violence, and everyone near the crime scene needs to use caution immediately. To be clear, we are definitely not saying that "NARCOTICS" (for example selling or possessing drugs) or "OFFENSE INVOLVING CHILDREN" (mostly child abuse in our dataset) are categorized as 0 because they are not severe in nature. In other words, our definition of "severe" here is more like ongoing crime that can physically harm everyone nearby versus crimes that are like "dirty secrets" of the criminals.

In [8]:
severe_crime_list=["ARSON", "ASSAULT", "BATTERY", "CRIM SEXUAL ASSAULT", "CRIMINAL DAMAGE", "CRIMINAL TRESPASS", "HOMICIDE", "ROBBERY"]
data_small["severe"]=np.where(data_small['Primary Type'].isin(severe_crime_list), 1, 0)
data_small.head(5)

,ID,Date,Block,IUCR,Primary Type,Description,Location Description,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location,severe
0,10313623,11/15/2015 11:51:00 PM,069XX S DAMEN AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,7,17,67,16,1164219,1858818,2015,41.768214,-87.673617,"(41.768213543, -87.673617146)",0
1,10313594,11/15/2015 11:46:00 PM,041XX W 47TH ST,0560,ASSAULT,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),8,14,57,08A,1149632,1873148,2015,41.807832,-87.726715,"(41.807831884, -87.726715402)",1
2,10313635,11/15/2015 11:40:00 PM,012XX N HONORE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,14,1,24,08B,1163925,1908232,2015,41.903817,-87.673303,"(41.903816925, -87.673303026)",1
3,10313604,11/15/2015 11:40:00 PM,059XX S CALUMET AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,ALLEY,2,20,40,14,1179499,1865673,2015,41.786689,-87.617400,"(41.786689088, -87.617399787)",1
4,10314701,11/15/2015 11:30:00 PM,026XX N BOSWORTH AVE,0810,THEFT,OVER $500,STREET,19,32,7,06,1165540,1917725,2015,41.929832,-87.667100,"(41.929832059, -87.667099876)",0


Another important feature for classification is the field of "Location Description". We can imagine robbery is more likely to happen on a street compared with a restaurant, while deceptive practice is probably going to happen at a store instead of at home. Let's see how many types of location they have.

In [9]:
print len(data_small.groupby("Location Description")["Location Description"].count().index.values)

134


There are 134 types of locations, which is a lot. Again, there are locations that are extremely rare. Let's see what the top ones are.

In [10]:
datagb_location=data_small.groupby("Location Description")["Location Description"].count()
datagb_location.order(ascending=False, inplace=True)
datagb_location.head(50)

Location Description
STREET                               332327
RESIDENCE                            202081
APARTMENT                            172835
SIDEWALK                             171287
OTHER                                 49367
PARKING LOT/GARAGE(NON.RESID.)        42283
ALLEY                                 32655
RESIDENTIAL YARD (FRONT/BACK)         30862
SCHOOL, PUBLIC, BUILDING              27824
SMALL RETAIL STORE                    26881
RESIDENCE-GARAGE                      26704
RESIDENCE PORCH/HALLWAY               25535
VEHICLE NON-COMMERCIAL                23904
RESTAURANT                            23600
DEPARTMENT STORE                      19847
GROCERY FOOD STORE                    15867
GAS STATION                           14825
PARK PROPERTY                         12527
BAR OR TAVERN                          9816
CTA PLATFORM                           8518
CTA TRAIN                              7262
VACANT LOT/LAND                        7105
COMMERCIAL 

Again we can see some most popular locations account for most of crime scenes. To avoid the curse of dimensionality (since we are going to convert individual location to indicator variable), we will just drop crimes that happen at extremely unlikely locations, and only keep crimes that happen at the top 25 locations. We can see below even after two rounds of filtering, we still keep around 85% of the crimes. Therefore, we are not over-filtering our data.

In [11]:
location_list=datagb_location.index.values[0:25].tolist()
data_small=data_small[data_small["Location Description"].isin(location_list)]
data_small.shape

(1304026, 18)

We will now check some other features that can be used for the classfier. Police district and community area are the two features that can be important, since we can expect bad neighbourhood may have higher chance to have crimes that are more violent.

In [12]:
print data_small.groupby("District")["District"].count()
print data_small.groupby("Community Area")["Community Area"].count()

District
1     44940
2     53934
3     68218
4     78864
5     58822
6     79018
7     79635
8     89160
9     66075
10    59534
11    92774
12    59828
14    48309
15    60785
16    37606
17    37955
18    52187
19    58622
20    20670
22    42944
24    36532
25    77560
31       52
Name: District, dtype: int64
Community Area
0        12
1     18444
2     16038
3     17993
4      8564
5      7342
6     28439
7     19229
8     40287
9      1234
10     5376
11     5158
12     2161
13     3988
14    11850
15    16548
16    15374
17     7720
18     2889
19    24417
20     8276
21    12038
22    26080
23    44354
24    37222
25    88909
26    28397
27    26271
28    34016
29    40688
      ...  
48     7420
49    35856
50     4678
51     8941
52     6895
53    22232
54     4508
55     2861
56     8599
57     4989
58    13606
59     5406
60     8337
61    27081
62     5161
63    13072
64     4792
65    10152
66    33737
67    39357
68    35833
69    33729
70    11551
71    38476
72     4333

We can see there are 77 community areas, and 22 districts (no 13, 21, and 23; the district 31 can be a typo). This is reasonable because one police district can cover multiple areas. As a starting point, we can use the district as a location factor (but we can also use community area as well). Of course we need to drop typos.

In [13]:
datagb_destrict=data_small.groupby("District")["District"].count()
district_list=datagb_destrict.index.values[0:22].tolist()
data_small=data_small[data_small["District"].isin(district_list)]
data_small.shape

(1303972, 18)

To avoid district names are wrongly used as integers, we will simply add a D to all the district name. This will be helpful for the later step (for example when individual district is converted to dummy variable).

In [14]:
#reference: http://stackoverflow.com/questions/20025882/append-string-to-the-start-of-each-value-in-a-said-column-of-a-pandas-dataframe
data_small["District"]='D'+data_small['District'].astype(str)
data_small.head(2)

,ID,Date,Block,IUCR,Primary Type,Description,Location Description,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location,severe
0,10313623,11/15/2015 11:51:00 PM,069XX S DAMEN AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,D7.0,17,67,16,1164219,1858818,2015,41.768214,-87.673617,"(41.768213543, -87.673617146)",0
1,10313594,11/15/2015 11:46:00 PM,041XX W 47TH ST,0560,ASSAULT,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),D8.0,14,57,08A,1149632,1873148,2015,41.807832,-87.726715,"(41.807831884, -87.726715402)",1


Here comes the key part. In order to do classfication based on most package, it will be preferable to convert the categorical data into dummy variable (0 and 1). Some packages may not require that, but we convert it anyway. This can also be done using sklearn OneHotEncoder command (which also converts categorical data to 0 and 1).
http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

Here in our case, we will use the get_dummies method provided by pandas:
http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.get_dummies.html

pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False)
Convert categorical variable into dummy/indicator variables

In [15]:
#make a dummy variable for district and primary type of crime
dummydf=pd.get_dummies(data_small,columns=["Primary Type","District"])
#the get_dummies function will actually break the whole district column without preserving the original column.
#we will just make a copy here in case we need to use it in the future
dummydf=dummydf.join(data_small[["District","Primary Type"]])
print dummydf.shape
dummydf.head(2)

(1303972, 64)


,ID,Date,Block,IUCR,Description,Location Description,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location,severe,Primary Type_ARSON,Primary Type_ASSAULT,Primary Type_BATTERY,Primary Type_BURGLARY,Primary Type_CRIM SEXUAL ASSAULT,Primary Type_CRIMINAL DAMAGE,Primary Type_CRIMINAL TRESPASS,Primary Type_DECEPTIVE PRACTICE,Primary Type_GAMBLING,Primary Type_HOMICIDE,Primary Type_INTERFERENCE WITH PUBLIC OFFICER,Primary Type_INTIMIDATION,Primary Type_KIDNAPPING,Primary Type_LIQUOR LAW VIOLATION,Primary Type_MOTOR VEHICLE THEFT,Primary Type_NARCOTICS,Primary Type_OFFENSE INVOLVING CHILDREN,Primary Type_PROSTITUTION,Primary Type_PUBLIC PEACE VIOLATION,Primary Type_ROBBERY,Primary Type_SEX OFFENSE,Primary Type_STALKING,Primary Type_THEFT,Primary Type_WEAPONS VIOLATION,District_D1.0,District_D10.0,District_D11.0,District_D12.0,District_D14.0,District_D15.0,District_D16.0,District_D17.0,District_D18.0,District_D19.0,District_D2.0,District_D20.0,District_D22.0,District_D24.0,District_D25.0,District_D3.0,District_D4.0,District_D5.0,District_D6.0,District_D7.0,District_D8.0,District_D9.0,District,Primary Type
0,10313623,11/15/2015 11:51:00 PM,069XX S DAMEN AVE,1506,SOLICIT ON PUBLIC WAY,STREET,17,67,16,1164219,1858818,2015,41.768214,-87.673617,"(41.768213543, -87.673617146)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,D7.0,PROSTITUTION
1,10313594,11/15/2015 11:46:00 PM,041XX W 47TH ST,0560,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),14,57,08A,1149632,1873148,2015,41.807832,-87.726715,"(41.807831884, -87.726715402)",1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,D8.0,ASSAULT


Now we need to deal with the time data. For each date, I will segment the time into eight blocks of 3-hour, for example 0-3am, 3-6am, et al. We will then create indicator variable based on that. Of course, we can also treat each individual time point as a data point instead of binning all data points together (in that case, we can calculate an exact distance between two time points for KNN or other distance-based algorithms). The situation is actually similar for the location data: we can use the exact coordiates to calculate certain distance for classification, or we can simply use the police districts (now as indicator variable) which bin / cluster the coordiates together. In the current data cleanup step, I will save both the exact information and the "binned" information, but we will determine what information to use later depends on the exact strategies.

In [16]:
%%time
#reference: http://stackoverflow.com/questions/1759455/how-can-i-account-for-period-am-pm-with-datetime-strptime
#I will process everything in a new timedataf, and later join this timedataf back to our original dummydf
timedataf=dummydf.copy()
#We will use datetime to convert the AM PM to 24-hour time
from datetime import datetime
format = '%m/%d/%Y %I:%M:%S %p'
#convert the time to 24-hour time to make our lives easier
dummydf["time_24hour"]=dummydf.Date.apply(lambda row: datetime.strptime(row, format).strftime("%H:%M"))
#split into eight time blocks. Block 0 will be 0:00 to 3:00, block 3 will be 3:00 to 6:00... The final block will be block 21, meaning 21:00 to 24:00.
dummydf["Timeblock"]=dummydf.Date.apply(lambda row: str(3*int(int(datetime.strptime(row, format).strftime("%H"))/3)))
#Have a date without time. Will be helpful later for integrating other information such as weather.
dummydf['Date_no_time']=dummydf.Date.apply(lambda row: datetime.strptime(row, format).strftime("%Y%m%d"))
#Of course we need weekday info. We can convert it to dummy variable as well.
dummydf["Weekday"]=dummydf.Date.apply(lambda row: datetime.strptime(row, format).strftime("%A"))
dummydf=pd.get_dummies(dummydf,columns=["Timeblock","Weekday"])
dummydf.shape

Wall time: 4min 46s


We found a very good dataset of Chicago weather from national centers for environmental information (data recorded on Chicago Midway Airport). This dataset contains temperature information and other weather info that can be helpful for predicting crime type. For example, we are expecting that when weather is hot, the crime will be more likely to involve violent. Similarly, when weather is extremely cold, the crimes happening indoor may increase.

https://www.ncdc.noaa.gov/cdo-web/datasets/

In [17]:
data_weather=pd.read_csv("chicago_weather.csv",index_col=0)
data_weather.head(5)

,PRCP,TMAX,TMIN,WT01,WT05,WT02,WT08,WT03,WT10
20080101,0,-21,-149,1,-9999,-9999,-9999,-9999,-9999
20080102,0,-60,-160,1,-9999,-9999,-9999,-9999,-9999
20080103,0,-49,-171,-9999,-9999,-9999,-9999,-9999,-9999
20080104,0,17,-55,-9999,-9999,-9999,-9999,-9999,-9999
20080105,5,61,17,1,-9999,-9999,-9999,-9999,-9999


Please not that the temperature data is in tenths of degrees C. Therefore, we will divide the value by 10 to normal scale. Additionally, -9999 means 0 in this case. WT are just weather types as described below (not all types are recorded at this station).

WT01	Weather Type fog or ice fog or freezing fog (may include heavy fog)
WT02	Weather Type heavy fog or heaving freezing fog (not always distinguished from fog)
WT03	Weather Type thunder
WT04	Weather Type ice pellets or sleet or snow pellets or small hail 
WT05	Weather Type hail (may include small hail)
WT06	Weather Type glaze or rime 
WT07	Weather Type dust or volcanic ash or blowing dust or blowing sand or blowing obstruction
WT08	Weather Type smoke or haze 
WT09	Weather Type blowing or drifting snow
WT10	Weather Type tornado or waterspout or funnel cloud 
WT11	Weather Type high or damaging winds
WT12	Weather Type blowing spray
WT13	Weather Type mist
WT14	Weather Type drizzle
WT15	Weather Type freezing drizzle 
WT16	Weather Type rain (may include freezing rain or drizzle or freezing drizzle) 
WT17	Weather Type freezing rain 
WT18	Weather Type snow or snow pellets or snow grains or ice crystals
WT19	Weather Type unknown source of precipitation 
WT21	Weather Type ground fog 
WT22	Weather Type ice fog or freezing fog

In [18]:
data_weather["TMAX"]=0.1*data_weather["TMAX"]
data_weather["TMIN"]=0.1*data_weather["TMIN"]

def convert9999 (r):
    if r==-9999:
        r=0
    return r

data_weather["WT01"]=data_weather["WT01"].apply(convert9999)
data_weather["WT05"]=data_weather["WT05"].apply(convert9999)
data_weather["WT02"]=data_weather["WT02"].apply(convert9999)
data_weather["WT08"]=data_weather["WT08"].apply(convert9999)
data_weather["WT03"]=data_weather["WT03"].apply(convert9999)
data_weather["WT10"]=data_weather["WT10"].apply(convert9999)

In [19]:
data_weather.head(5)

,PRCP,TMAX,TMIN,WT01,WT05,WT02,WT08,WT03,WT10
20080101,0,-2.1,-14.9,1,0,0,0,0,0
20080102,0,-6.0,-16.0,1,0,0,0,0,0
20080103,0,-4.9,-17.1,0,0,0,0,0,0
20080104,0,1.7,-5.5,0,0,0,0,0,0
20080105,5,6.1,1.7,1,0,0,0,0,0


In [21]:
%%time
dummydf["Precipitation"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].PRCP)
dummydf["TMAX"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].TMAX)
dummydf["TMIN"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].TMIN)
dummydf["WT01"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].WT01)
dummydf["WT05"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].WT05)
dummydf["WT02"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].WT02)
dummydf["WT08"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].WT08)
dummydf["WT03"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].WT03)
dummydf["WT10"]=dummydf["Date_no_time"].apply(lambda row: data_weather.loc[int(row)].WT10)

Wall time: 47min 37s


In [22]:
dummydf.tail(5)

,ID,Date,Block,IUCR,Description,Location Description,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location,severe,Primary Type_ARSON,Primary Type_ASSAULT,Primary Type_BATTERY,Primary Type_BURGLARY,Primary Type_CRIM SEXUAL ASSAULT,Primary Type_CRIMINAL DAMAGE,Primary Type_CRIMINAL TRESPASS,Primary Type_DECEPTIVE PRACTICE,Primary Type_GAMBLING,Primary Type_HOMICIDE,Primary Type_INTERFERENCE WITH PUBLIC OFFICER,Primary Type_INTIMIDATION,Primary Type_KIDNAPPING,Primary Type_LIQUOR LAW VIOLATION,Primary Type_MOTOR VEHICLE THEFT,Primary Type_NARCOTICS,Primary Type_OFFENSE INVOLVING CHILDREN,Primary Type_PROSTITUTION,Primary Type_PUBLIC PEACE VIOLATION,Primary Type_ROBBERY,Primary Type_SEX OFFENSE,Primary Type_STALKING,Primary Type_THEFT,Primary Type_WEAPONS VIOLATION,District_D1.0,District_D10.0,District_D11.0,District_D12.0,District_D14.0,District_D15.0,District_D16.0,District_D17.0,District_D18.0,District_D19.0,District_D2.0,District_D20.0,District_D22.0,District_D24.0,District_D25.0,District_D3.0,District_D4.0,District_D5.0,District_D6.0,District_D7.0,District_D8.0,District_D9.0,District,Primary Type,time_24hour,Date_no_time,Timeblock_0,Timeblock_12,Timeblock_15,Timeblock_18,Timeblock_21,Timeblock_3,Timeblock_6,Timeblock_9,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Precipitation,TMAX,TMIN,WT01,WT05,WT02,WT08,WT03,WT10
1499993,7867604,12/24/2010 06:00:00 PM,021XX N ELSTON AVE,0840,FINANCIAL ID THEFT: OVER $300,DEPARTMENT STORE,32,22,06,1164535,1914261,2010,41.920348,-87.670891,"(41.920348003, -87.670891311)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,D14.0,THEFT,18:00,20101224,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,20,0,-2.1,1,0,0,1,0,0
1499994,7861523,12/24/2010 06:00:00 PM,096XX S WOODLAWN AVE,1320,TO VEHICLE,STREET,8,50,14,1185855,1841327,2010,41.719734,-87.594862,"(41.719733681, -87.594861812)",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,D5.0,CRIMINAL DAMAGE,18:00,20101224,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,20,0,-2.1,1,0,0,1,0,0
1499996,7861185,12/24/2010 06:00:00 PM,079XX S SAGINAW AVE,1320,TO VEHICLE,PARKING LOT/GARAGE(NON.RESID.),7,46,14,1195182,1853032,2010,41.751629,-87.560315,"(41.751628575, -87.560315225)",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,D4.0,CRIMINAL DAMAGE,18:00,20101224,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,20,0,-2.1,1,0,0,1,0,0
1499997,7861634,12/24/2010 06:00:00 PM,031XX N KILBOURN AVE,0610,FORCIBLE ENTRY,RESIDENCE,31,20,05,1145874,1920334,2010,41.937388,-87.739301,"(41.937387982, -87.739301227)",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,D25.0,BURGLARY,18:00,20101224,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,20,0,-2.1,1,0,0,1,0,0
1499998,7861734,12/24/2010 06:00:00 PM,043XX N LINCOLN AVE,0610,FORCIBLE ENTRY,RESIDENCE PORCH/HALLWAY,47,5,05,1161238,1928639,2010,41.959871,-87.682604,"(41.959871343, -87.682604081)",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,D19.0,BURGLARY,18:00,20101224,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,20,0,-2.1,1,0,0,1,0,0


In [23]:
dummydf.to_csv("chicago_small_clean.csv", index=False)

In [24]:
dummydf.shape

(1303972, 90)

After all the clean up, we have more than 85% records kept from December 2010 to Nov

# Reference point to delete: Jason starting cleaning from here

The purpose of the few steps below is to calculate the distance of each crime to the closest police station. As we can imagine, it is not very likely that a severe crime like robbery is going to happen right in front of a police station.

Since we have the longitude / latitude data of each crime, we will simply calculate the distances between the crime scence and all the police stations in Chicago. This way we can find out the distance of the crime to the closest police station.

In [145]:
chicago =pd.read_csv("chicago_small_clean.csv")
chicago

,ID,Date,Block,IUCR,Description,Location Description,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location,severe,Primary Type_ARSON,Primary Type_ASSAULT,Primary Type_BATTERY,Primary Type_BURGLARY,Primary Type_CRIM SEXUAL ASSAULT,Primary Type_CRIMINAL DAMAGE,Primary Type_CRIMINAL TRESPASS,Primary Type_DECEPTIVE PRACTICE,Primary Type_GAMBLING,Primary Type_HOMICIDE,Primary Type_INTERFERENCE WITH PUBLIC OFFICER,Primary Type_INTIMIDATION,Primary Type_KIDNAPPING,Primary Type_LIQUOR LAW VIOLATION,Primary Type_MOTOR VEHICLE THEFT,Primary Type_NARCOTICS,Primary Type_OFFENSE INVOLVING CHILDREN,Primary Type_PROSTITUTION,Primary Type_PUBLIC PEACE VIOLATION,Primary Type_ROBBERY,Primary Type_SEX OFFENSE,Primary Type_STALKING,Primary Type_THEFT,Primary Type_WEAPONS VIOLATION,District_D1.0,District_D10.0,District_D11.0,District_D12.0,District_D14.0,District_D15.0,District_D16.0,District_D17.0,District_D18.0,District_D19.0,District_D2.0,District_D20.0,District_D22.0,District_D24.0,District_D25.0,District_D3.0,District_D4.0,District_D5.0,District_D6.0,District_D7.0,District_D8.0,District_D9.0,District,Primary Type,time_24hour,Date_no_time,Timeblock_0,Timeblock_12,Timeblock_15,Timeblock_18,Timeblock_21,Timeblock_3,Timeblock_6,Timeblock_9,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Precipitation,TMAX,TMIN,WT01,WT05,WT02,WT08,WT03,WT10
0,10313623,11/15/2015 11:51:00 PM,069XX S DAMEN AVE,1506,SOLICIT ON PUBLIC WAY,STREET,17,67,16,1164219,1858818,2015,41.768214,-87.673617,"(41.768213543, -87.673617146)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,D7.0,PROSTITUTION,23:51,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
1,10313594,11/15/2015 11:46:00 PM,041XX W 47TH ST,0560,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),14,57,08A,1149632,1873148,2015,41.807832,-87.726715,"(41.807831884, -87.726715402)",1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,D8.0,ASSAULT,23:46,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
2,10313635,11/15/2015 11:40:00 PM,012XX N HONORE ST,0486,DOMESTIC BATTERY SIMPLE,STREET,1,24,08B,1163925,1908232,2015,41.903817,-87.673303,"(41.903816925, -87.673303026)",1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,D14.0,BATTERY,23:40,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
3,10313604,11/15/2015 11:40:00 PM,059XX S CALUMET AVE,1320,TO VEHICLE,ALLEY,20,40,14,1179499,1865673,2015,41.786689,-87.617400,"(41.786689088, -87.617399787)",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,D2.0,CRIMINAL DAMAGE,23:40,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
4,10314701,11/15/2015 11:30:00 PM,026XX N BOSWORTH AVE,0810,OVER $500,STREET,32,7,06,1165540,1917725,2015,41.929832,-87.667100,"(41.929832059, -87.667099876)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,D19.0,THEFT,23:30,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
5,10313645,11/15/2015 11:30:00 PM,003XX S SPRINGFIELD AVE,1320,TO VEHICLE,STREET,24,26,14,1150451,1897935,2015,41.875835,-87.723066,"(41.875834691, -87.723065513)",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,D11.0,CRIMINAL DAMAGE,23:30,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
6,10313958,11/15/2015 11:30:00 PM,107XX S LA SALLE ST,0810,OVER $500,STREET,34,49,06,1177179,1833692,2015,41.698982,-87.626869,"(41.698982009, -87.626868724)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,D5.0,THEFT,23:30,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0
7,10313610,11/15/2015 11:26:00 PM,055XX W GRAND AVE,2022,POSS: COCAINE,STREET,29,19,18,1138788,1913480,2015,41.918712,-87.765511,"(41.918711651, -87.7655106

In [146]:
# drop all the NaNs in the dataframe
chicago = chicago.dropna()
chicago.shape

(1293862, 90)

In [147]:
# See which police station is closest to the crime of interest...
# Scrap addresses from all Chicago police stations. There are 21 of them. Website scrapping from: https://portal.chicagopolice.org/portal/page/portal/ClearPath/Communities/Districts
# http://zevross.com/blog/2014/05/16/using-the-python-library-beautifulsoup-to-extract-data-from-a-webpage-applied-to-world-cup-rankings/
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
import requests
import time
table = []
police=requests.get("https://portal.chicagopolice.org/portal/page/portal/ClearPath/Communities/Districts")
soup = BeautifulSoup(police.text, "html.parser") 

for hit in soup.findAll("font", {"class": "descriptionid1siteid277"}):
    table.append(hit.contents)

# delete first and last rows
del table[0]
del table[22]

# flatten list of list
table1 = sum(table, [])
# clean up \r\n
table1 = [s.replace("\r\n", "") for s in table1]
# delete phone number by splitting at the dash
final_table = []
for i in table1:
    final_table.append(i.split("-")[0])

In [148]:
# Using Google's geocode REST api to get Police stations' longitude and latitude
# http://stackoverflow.com/questions/25888396/how-to-get-latitude-longitude-with-python
lati = []
longi = []
for i in final_table:
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+i)
    resp_json_payload = response.json()
    #print resp_json_payload['results'][0]['geometry']['location']
    lati.append(float(resp_json_payload['results'][0]['geometry']['location']['lat']))
    longi.append(float(resp_json_payload['results'][0]['geometry']['location']['lng']))
    time.sleep(1)

In [149]:
police_stations_loc =zip(lati, longi)

In [150]:
# Scrap police district number
district_table = []
district_table_final = []
for i in soup.findAll("font", {"class": "titleorimageid1siteid277"}):
    district_table.append(i.contents)
# delete first three rows since they are not police districts
del district_table[0:3]
district_table1 = sum(district_table, [])
for i in district_table1:
    district_table_final.append(i.split("-")[0])

In [151]:
# create new dataframe for police districts and their latitude and longitutide coordinates
police_df = pd.DataFrame(dict(latitude = lati,longitude = longi, District_police = district_table_final, pol_location = police_stations_loc))
police_df.head()

,District_police,latitude,longitude,pol_location
0,1st District,41.858226,-87.627522,"(41.8582258, -87.6275221)"
1,2nd District,41.801751,-87.630696,"(41.8017506, -87.6306963)"
2,3rd District,41.767015,-87.605638,"(41.7670154, -87.6056378)"
3,4th District,41.708022,-87.568174,"(41.7080224, -87.5681739)"
4,5th District,41.692176,-87.603381,"(41.6921756, -87.6033809)"


In [152]:
# small sample crime locations for debuging 
new_chicago = chicago[0:100]

In [153]:
%time
# change string into float
#new_chicago["Location"] = zip(new_chicago["Latitude"], new_chicago["Longitude"])
chicago["Location"] = zip(chicago["Latitude"], chicago["Longitude"])

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.06 µs


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [154]:
# police station location
station_array = np.array(police_stations_loc)
#station_array

## Calculating distance between particular crime location and its closest police station
We will use the Harvesine Formula to calculate the closest police station to each of the crime scenes in our dataset:
### $\operatorname{hav}\left(\frac{d}{r}\right) = \operatorname{hav}(\phi_2 - \phi_1) + \cos(\phi_1) \cos(\phi_2)\operatorname{hav}(\lambda_2-\lambda_1)$

where,
* ''hav'' is the haversine function:
$\operatorname{hav}(\theta)=\sin^2\left(\frac{\theta}{2}\right)=\frac{1-\cos(\theta)}{2}$
* ''d'' is the distance between the two points along a great circle of the sphere; 
* ''r'' is the radius of the sphere,
* $\phi_1, \phi_2$: latitude of point 1 and latitude of point 2
* $\lambda_1, \lambda_2$: longitude of point 1 and longitude of point 2

In [156]:
%time
# find minimal distance between crime location and any of the 21 police stations
# Reference: http://stackoverflow.com/questions/6656475/python-speeding-up-geographic-comparison
from math import radians, sin, cos, asin, sqrt, pi, atan2
import itertools
from timeit import Timer

distance = []
earth_radius_miles = 3956.0
def get_shortest_in(needle, haystack):
# needle is a single (lat,long) tuple. haystack is a numpy array to find the point in that has the shortest distance to needle  
    dlat = np.radians(haystack[:,0]) - radians(needle[0])
    dlon = np.radians(haystack[:,1]) - radians(needle[1])
    a = np.square(np.sin(dlat/2.0)) + cos(radians(needle[0])) * np.cos(np.radians(haystack[:,0])) * np.square(np.sin(dlon/2.0))
    great_circle_distance = 2 * np.arcsin(np.minimum(np.sqrt(a), np.repeat(1, len(a))))
    d = earth_radius_miles * great_circle_distance
    return np.min(d)

#x = new_chicago["Location"]
x = chicago["Location"]
y = station_array

def donumpy():
    get_shortest_in(x, y)

for i in x:
    distance.append(get_shortest_in(i, y))
#print 'numpy distance =', get_shortest_in(x, y), 'time =',
#print Timer("donumpy()", "from __main__ import donumpy").timeit(100)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [159]:
# distance in miles
chicago["closest_station"] = distance

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [160]:
chicago.to_csv("Jason_clean.csv", index=False)

In [161]:
chicago.shape

(1293862, 91)

In [9]:
smalldf = pd.read_csv("Jason_clean.csv")

### Xiaoji continue with cleaning

Given a location / coordinate, there are other relevant information such as income, education level and population structure of the neighbourhood. Though we may simply treat each single ward / community area / police district as an indicator variable that incorporates all the information of a given region together (which we do in the case of police district), we may fail to reveal interesting associations with individual features of the region, such as local income versus crime type. As a result, we include several columns of continuous variables describing income / education level / population structure based on the "Community Area". The data is also provided by the government of Chicago.

To avoid the duplication of information, we will not use community area as a categorical feature (indicator variable) in our later analysis (since we have now generated several continuous features based on this feature). We will still indlude the police district as a categorical feature (indicator variable), since the information provided by the police district feature does not overlap 100% with those features generated from community area (though correlation exists, since a police district covers several community areas). For example, two locations belong to two police districts may have similar income but different types of crime, depending on the man power or focus of the police district. Therefore, we will use police district information as a categorical feature (of course, we can also break this categorical feature into several small features, such as number of police per area / population, but those information will be hard to find).

In [23]:
inc_edu_age = pd.read_csv("income_education_hardship.csv",index_col=0)
inc_edu_age.head(5)

,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER_CAPITA_INCOME,HARDSHIP INDEX
1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [24]:
smalldf.head()

,ID,Date,Block,IUCR,Description,Location Description,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location,severe,Primary Type_ARSON,Primary Type_ASSAULT,Primary Type_BATTERY,Primary Type_BURGLARY,Primary Type_CRIM SEXUAL ASSAULT,Primary Type_CRIMINAL DAMAGE,Primary Type_CRIMINAL TRESPASS,Primary Type_DECEPTIVE PRACTICE,Primary Type_GAMBLING,Primary Type_HOMICIDE,Primary Type_INTERFERENCE WITH PUBLIC OFFICER,Primary Type_INTIMIDATION,Primary Type_KIDNAPPING,Primary Type_LIQUOR LAW VIOLATION,Primary Type_MOTOR VEHICLE THEFT,Primary Type_NARCOTICS,Primary Type_OFFENSE INVOLVING CHILDREN,Primary Type_PROSTITUTION,Primary Type_PUBLIC PEACE VIOLATION,Primary Type_ROBBERY,Primary Type_SEX OFFENSE,Primary Type_STALKING,Primary Type_THEFT,Primary Type_WEAPONS VIOLATION,District_D1.0,District_D10.0,District_D11.0,District_D12.0,District_D14.0,District_D15.0,District_D16.0,District_D17.0,District_D18.0,District_D19.0,District_D2.0,District_D20.0,District_D22.0,District_D24.0,District_D25.0,District_D3.0,District_D4.0,District_D5.0,District_D6.0,District_D7.0,District_D8.0,District_D9.0,District,Primary Type,time_24hour,Date_no_time,Timeblock_0,Timeblock_12,Timeblock_15,Timeblock_18,Timeblock_21,Timeblock_3,Timeblock_6,Timeblock_9,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday,Precipitation,TMAX,TMIN,WT01,WT05,WT02,WT08,WT03,WT10,closest_station,House_crowded,House_below_poverty,Unemployed,Without_high_school,Under18_over64,Hardship
0,10313623,11/15/2015 11:51:00 PM,069XX S DAMEN AVE,1506,SOLICIT ON PUBLIC WAY,STREET,17,67,16,1164219,1858818,2015,41.768214,-87.673617,"(41.768213543000002, -87.673617145999998)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,D7.0,PROSTITUTION,23:51,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0,1.022147,4.8,34.4,35.9,26.3,40.7,89
1,10313594,11/15/2015 11:46:00 PM,041XX W 47TH ST,0560,SIMPLE,PARKING LOT/GARAGE(NON.RESID.),14,57,08A,1149632,1873148,2015,41.807832,-87.726715,"(41.807831884000002, -87.726715402000011)",1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,D8.0,ASSAULT,23:46,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0,2.182132,8.5,14.1,16.5,35.9,39.2,67
2,10313635,11/15/2015 11:40:00 PM,012XX N HONORE ST,0486,DOMESTIC BATTERY SIMPLE,STREET,1,24,08B,1163925,1908232,2015,41.903817,-87.673303,"(41.903816925000001, -87.673303025999999)",1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,D14.0,BATTERY,23:40,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0,1.528224,2.3,14.7,6.6,12.9,21.7,10
3,10313604,11/15/2015 11:40:00 PM,059XX S CALUMET AVE,1320,TO VEHICLE,ALLEY,20,40,14,1179499,1865673,2015,41.786689,-87.617400,"(41.786689088000003, -87.617399787000011)",1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,D2.0,CRIMINAL DAMAGE,23:40,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0,1.244959,5.6,42.1,28.6,25.4,42.8,88
4,10314701,11/15/2015 11:30:00 PM,026XX N BOSWORTH AVE,0810,OVER $500,STREET,32,7,06,1165540,1917725,2015,41.929832,-87.667100,"(41.929832058999999, -87.667099876000009)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,D19.0,THEFT,23:30,20151115,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,18.9,6.7,0,0,0,0,0,0,1.460247,0.8,12.3,5.1,3.6,21.5,2


In [27]:
%%time
#smalldf=smalldf[smalldf["Community Area"]!=0]
#smalldf["House_crowded"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)]["PERCENT OF HOUSING CROWDED"])
#smalldf["House_below_poverty"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)]["PERCENT HOUSEHOLDS BELOW POVERTY"])
#smalldf["Unemployed"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)]["PERCENT AGED 16+ UNEMPLOYED"])
#smalldf["Without_high_school"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)]["PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA"])
#smalldf["Under18_over64"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)]["PERCENT AGED UNDER 18 OR OVER 64"])
smalldf["Income"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)].iloc[6])
#smalldf["Hardship"]=smalldf["Community Area"].apply(lambda row: inc_edu_age.loc[int(row)]["HARDSHIP INDEX"])

Wall time: 3min 18s


Final cleanup of "location description" feature: we have "location description" column telling us exactly where the crime happened (e.g. on street or in an apartment). We have previously keep top 25 locations, and we will now split this categorical feature into dummy variables. Note that "OTHER" is in the list, and we will throw it away. This is the last step of cleanup for now.

In [28]:
smalldf.head()
print smalldf.shape
smalldf=smalldf[smalldf["Location Description"]!="OTHER"]
smalldf=pd.get_dummies(smalldf,columns=["Location Description"])
print smalldf.shape
smalldf.to_csv("chicago_final_clean.csv", index=False)

(1293857, 98)
(1245208, 121)
